#Решение

##загрузка данных

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#train = pd.read_parquet('train_with_tree_category.parquet')
train = pd.read_parquet('train.parquet')
test = pd.read_parquet('test.parquet')
categories_tree = pd.read_csv('categories_tree.csv')

In [ ]:
categories_tree.head()

,id,title,parent_id
0,1,Все категории,0
1,114,Урбеч,1913
2,115,Варенье и джемы,328
3,128,Сухие завтраки,2475
4,131,Масла,2475


In [ ]:
train.head()

,id,title,short_description,name_value_characteristics,rating,feedback_quantity,category_id
0,1267423,Muhle Manikure Песочные колпачки для педикюра ...,Muhle Manikure Колпачок песочный шлифовальный ...,None,0.000000,0,2693
1,128833,"Sony Xperia L1 Защитное стекло 2,5D",None,None,4.666667,9,13408
2,569924,"Конверт для денег Прекрасная роза, 16,5 х 8 см","Конверт для денег «Прекрасная роза», 16,5 × 8 см",None,5.000000,6,11790
3,1264824,Серьги,None,None,0.000000,0,14076
4,1339052,Наклейки на унитаз для туалета на крышку бачок...,"Водостойкая, интересная наклейка на унитаз раз...",None,0.000000,0,12401


In [ ]:
test.head()

,id,title,short_description,name_value_characteristics,rating,feedback_quantity
0,1070974,Браслет из натуральных камней LOTUS,None,None,0.000000,0
1,450413,Fusion Life - Шампунь для сухих и окрашенных в...,None,None,4.333333,6
2,126857,"Микрофон для ПК jack 3,5мм всенаправленный","универсальный 3,5 мм микрофон запишет ваш звук",None,3.708333,24
3,1577569,Серьги гвоздики сердце,Серьги гвоздики сердце,None,0.000000,0
4,869328,"Чёрно-красная стильная брошь ""Тюльпаны"" из акр...",Стильная и яркая брошь ручной работы! Великоле...,None,0.000000,0


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283452 entries, 0 to 283451
Data columns (total 7 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          283452 non-null  int64  
 1   title                       283452 non-null  object 
 2   short_description           133130 non-null  object 
 3   name_value_characteristics  50360 non-null   object 
 4   rating                      283452 non-null  float64
 5   feedback_quantity           283452 non-null  int64  
 6   category_id                 283452 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 15.1+ MB


In [ ]:
train = train.fillna('none')
train[['title', 'short_description', 'name_value_characteristics']] = train[['title', 'short_description', 'name_value_characteristics']].astype(str)

In [ ]:
test = test.fillna('none')
test[['title', 'short_description', 'name_value_characteristics']] = test[['title', 'short_description', 'name_value_characteristics']].astype(str)

In [ ]:
#получение дерева признаков
def get_tree(id, path_list):
  parent_id = categories_tree[categories_tree['id'] == id]['parent_id'].values[0]
  
  if parent_id  > 0:
    path_list.append(parent_id)
    return get_tree(parent_id, path_list)
  else:
    if len(path_list) != 6:
      path_list.append(0)
    
    return path_list

#построение features для category
def build_features(id):
  path_list = get_tree(id, [id])

  path_list.reverse()

  return path_list + [-1] * (6 - len(path_list))

##предобработка

In [ ]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 8.2 MB 7.5 MB/s 
     |████████████████████████████████| 371 kB 62.2 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=867888 sha256=3c7546b6711b794d9c3942ed36c09cd6b503e66f73742bfce0d8edb48759a514
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [ ]:
import pymorphy2
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

morph = pymorphy2.MorphAnalyzer()
stop_words = set(stopwords.words('russian') + stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def preprocessing(sentence):
  rez = ''
  sentence = re.sub(r'[^\w\s]', '', sentence)
  words = word_tokenize(sentence)

  for i in word_tokenize(sentence):
    if i not in stop_words:
      rez += morph.parse(i)[0].normal_form + ' '
  
  return rez

построение дерева признаков для каждого id в таблице


!выполняется довольно долго, можно пропустить, если в начале использовать вместо train.parquet - train_with_tree_category.parquet

In [ ]:
train[['l0', 'l1', 'l2', 'l3', 'l4', 'l5']] = [-1, -1, -1, -1, -1, -1]
train = train[~train['category_id'].isin([13182, 12652])]

features = []
for i in train.index:
  features.append(build_features(train.loc[i].category_id))

train[['l0', 'l1', 'l2', 'l3', 'l4', 'l5']] = features
#train.to_parquet('train_with_tree_category.parquet')

разобъем дерево признаков на уровни

In [ ]:
'''
l0 = categories_tree[categories_tree['parent_id'] == 0]['id']
l1 = categories_tree[categories_tree['parent_id'].isin(l0)]['id']
l2 = categories_tree.loc[categories_tree['parent_id'].isin(l1)]['id']
l3 = categories_tree.loc[categories_tree['parent_id'].isin(l2)]['id']
l4 = categories_tree.loc[categories_tree['parent_id'].isin(l3)]['id']
l5 = categories_tree.loc[categories_tree['parent_id'].isin(l4)]['id']

с лемматизацией работает хуже, поэтому только удаление стоп слов

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import hstack

#vectorizer = CountVectorizer(preprocessor=preprocessing)
vectorizer = CountVectorizer(stop_words=stop_words)
title_vec = vectorizer.fit_transform(train['title'] + ' ' + train['short_description'] + ' ' + train['name_value_characteristics'])

##Выбор и обучение моделей

разбиение данных и поиск наилучших параметров для моделей разных уровней

у catboost сильно выше f1 макроусреднение

In [ ]:
!pip install catboost

In [ ]:
'''
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

for i in range(6):
  x_train, x_test, y_train, y_test = train_test_split(
      title_vec,
      train['l' + str(i)],
      test_size=0.1,
      stratify=train['l' + str(i)],
      random_state=33
  )

  parameters = {'alpha':np.arange(0, 1.5, 0.05), 'fit_prior':[True, False]}

  best_params = GridSearchCV(MultinomialNB(), parameters).fit(x_train, y_train).best_params_

  predicted = clf.predict(x_test)
  print(best_params)
  print(f1_score(y_test, predicted, average='weighted'))
  print(f1_score(y_test, predicted, average='macro'))
  print(f1_score(y_test, predicted, average='micro'), end='\n\n')

In [ ]:
'''
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier

for i in range(3):
  x_train, x_test, y_train, y_test = train_test_split(
      title_vec,
      train['l' + str(i)],
      test_size=0.1,
      stratify=train['l' + str(i)],
      random_state=33
  )

  parameters = {'max_depth': np.arange(2, 10, 2), 'learning_rate':np.arange(0.01, 0.5, 0.05), 'bootstrap_type' : ['Bayesian', 'Bernoulli']}

  best_params = GridSearchCV(CatBoostClassifier(), parameters).fit(x_train, y_train).best_params_

  predicted = clf.predict(x_test)
  print(best_params)
  print(f1_score(y_test, predicted, average='weighted'))
  print(f1_score(y_test, predicted, average='macro'))
  print(f1_score(y_test, predicted, average='micro'), end='\n\n')

параметры для моделей искались с помощью GridSearchCV

In [ ]:
params_for_models = [{'alpha' : 1e-06, 'fit_prior' : True},
                     {'alpha' : 2.2, 'fit_prior' : True},
                     {'alpha' : 0.2, 'fit_prior' : True},
                     {'alpha' : 0.15, 'fit_prior' : True},
                     {'alpha' : 0.1, 'fit_prior' : False},
                     {'alpha' : 0.08, 'fit_prior' : False}]

In [ ]:
def cat_model(iterations=200, lr=0.5, max_depth=8):
  return CatBoostClassifier(
      iterations=iterations,
      learning_rate=lr,
      max_depth=max_depth,
      random_seed=33,
      #task_type='GPU'
  )

def multi_nb(params):
  return MultinomialNB(**params)

не получилось обучить для 4-6 уровенй catboost

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from catboost import CatBoostClassifier

tiered_models = []

tiered_models.append(cat_model(200, 0.08, 6))
tiered_models.append(cat_model(200, 0.08, 6))
tiered_models.append(cat_model(200, 0.08, 6))
tiered_models.append(multi_nb(params_for_models[3]))
tiered_models.append(multi_nb(params_for_models[4]))
tiered_models.append(multi_nb(params_for_models[5]))

In [ ]:
for i in range(6):
  tiered_models[i].fit(title_vec, train['l' + str(i)])

In [ ]:
title_vec_test = vectorizer.transform(test['title'] + test['short_description'] + test['name_value_characteristics'])

In [ ]:
for i in range(6):
  test['l' + str(i)] = tiered_models[i].predict(title_vec_test)

In [ ]:
test.head()

,id,title,short_description,name_value_characteristics,rating,feedback_quantity,l0,l1,l2,l3,l4,l5
0,1070974,Браслет из натуральных камней LOTUS,none,none,0.000000,0,0,1,10003,10023,10177,11574
1,450413,Fusion Life - Шампунь для сухих и окрашенных в...,none,none,4.333333,6,0,1,10012,10165,10705,11878
2,126857,"Микрофон для ПК jack 3,5мм всенаправленный","универсальный 3,5 мм микрофон запишет ваш звук",none,3.708333,24,0,1,10020,10144,11157,2803
3,1577569,Серьги гвоздики сердце,Серьги гвоздики сердце,none,0.000000,0,0,1,10003,10023,10104,13061
4,869328,"Чёрно-красная стильная брошь ""Тюльпаны"" из акр...",Стильная и яркая брошь ручной работы! Великоле...,none,0.000000,0,0,1,10003,10023,10104,12813


In [ ]:
test[test['l3'] == -1]

,id,title,short_description,name_value_characteristics,rating,feedback_quantity,l0,l1,l2,l3,l4,l5


In [ ]:
result = pd.DataFrame(columns=['id', 'predicted_category_id'])

result['id'] = test['id']
predicted_category = test['l5'].tolist()

for i in range(len(predicted_category)):
  if predicted_category[i] == -1:
    if test['l4'][i] != -1:
      predicted_category[i] = test['l4'][i]
    else:
      predicted_category[i] = test['l3'][i]

min(predicted_category)

2599

In [ ]:
result['predicted_category_id'] = predicted_category
#result.to_parquet('result.parquet')

##tests

Протестировал catboost/xgboost + rubert - смог нормально обучить только для первых 3 уровней, результаты были лучше чем у баесовского классификатора.

ИСопльзование rubert не дало сильного прироста f1

результаты получились хуже + модель очень долго обучается, а учитывая способ классификации (отдельно для каждого уровня), нужно очень много времени

In [ ]:
'''
!pip install transformers sentencepiece
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
model.cuda()

In [ ]:
'''
def embed_bert_cls(text, model=model, tokenizer=tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

x_train, y_train = [embed_bert_cls(i) for i in text], train['l0']
dataset = pd.read_parquet('berted.parquet')

In [ ]:
'''
from catboost import CatBoostClassifier

def model_catboost(iterations=50, lr=None, max_depth=4):
  return CatBoostClassifier(
      iterations=iterations, 
      learning_rate=lr,
      max_depth=max_depth,
      #task_type='GPU'
  )

def predict_tree(all_features, models, layers=6):
  res_tree = pd.DataFrame()
  predicted = models[0].predict(all_features)
  res_tree['l0'] = predicted

  for i in range(1, layers):
    predicted = models[i].predict(all_features)
    res_tree['l' + str(i)] = predicted
  
  return res_tree

In [ ]:
'''
models = [model_catboost(iterations=50), 
          model_catboost(iterations=35), 
          model_catboost(), 
          model_catboost(),
          model_catboost(),
          model_catboost()]

In [ ]:
'''
targets_name = ['l0', 'l1', 'l2', 'l3', 'l4', 'l5']
next_layer = x_train

for i in range(len(models)):
  models[i].fit(
      next_layer, train[targets_name[i]],
      )